### Входные данные
Форма для сдачи: https://goo.gl/forms/qptFQ1ukuCxeMEA63

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
#####Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
#####Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import collections

In [63]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in xrange(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        

def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [5]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

In [64]:
def example(stream):
    for value in stream:
        yield (value, value+10)
get_tuple_stream_mean(example(nightmare_stream()), 2)

[499910131.727, 499910141.727]

In [61]:
#easy_stream
def stream_turp1(stream):
    window = 1000
    arr = np.array([])
    for i, value in enumerate(stream):
        if (i+1)%window != 0:
            arr = np.append(arr, value)
        else:
            yield (np.mean(arr), np.std(arr))
            arr = np.array([])
            
def stream_turp2(stream):
    window = 1000
    arr = np.array([])
    for i, value in enumerate(stream):
        if (i+1)%window != 0:
            arr = np.append(arr, value)
        else:
            yield (np.min(arr), np.max(arr), np.median(arr))
            arr = np.array([])

In [68]:
print get_tuple_stream_mean(stream_turp1(nightmare_stream()), 2)
print get_tuple_stream_mean(stream_turp2(nightmare_stream()), 3)

[499900679.129, 286858684.089]
[10058404.654, 989994947.949, 499781214.956]
